<a href="https://colab.research.google.com/github/LeninGF/CoursesNotes/blob/main/InteligenciaArtificalGenerativa/Problems/transformersII-huggingface/LLM_IAG_2024B_PII_LeninGF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM pre entrenados Parte 2

Se estudia diferentes casos de aplicación como el QA extractivo y Generativo

Nombres: Lenin G. Falconí

Fecha: 2024-12-18

## QA Extractiva

In [1]:
!pip3 install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset
mlqa = load_dataset("xtreme",  name="MLQA.en.en")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/131k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/7.49M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/724k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/11590 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1148 [00:00<?, ? examples/s]

In [ ]:
mlqa

DatasetDict({
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11590
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 1148
    })
})

In [3]:
import torch
from transformers import AutoModelForQuestionAnswering
from transformers import AutoTokenizer
model_ckp = "deepset/minilm-uncased-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_ckp)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/107 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [4]:
# question, context, answer

In [5]:
def qa_extractive(model_ckp, question, context):
  model = AutoModelForQuestionAnswering.from_pretrained(model_ckp)
  with torch.no_grad():
    inputs = tokenizer(question, context, return_tensors="pt")
    outputs = model(**inputs)
  start_idx = torch.argmax(outputs.start_logits)
  end_idx = torch.argmax(outputs.end_logits)+1
  answer_span = inputs.input_ids[0, start_idx:end_idx+1]
  answer = tokenizer.decode(answer_span)
  print(f"Question: {question}\nAnswer:{answer}")


In [6]:
question = "How does Von Neuman's Computer Architecture use memory?"
context = "The Von Neuman Computer Architecture uses a main memory to store both the program and data. However, this causes a memory bottleneck because CPU speed is faster than access to memory"
answer = "It stores both the program instructions and data in a main memory"

In [7]:
question, context, answer

("How does Von Neuman's Computer Architecture use memory?",
 'The Von Neuman Computer Architecture uses a main memory to store both the program and data. However, this causes a memory bottleneck because CPU speed is faster than access to memory',
 'It stores both the program instructions and data in a main memory')

In [8]:
qa_extractive(model_ckp, question, context)

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

Some weights of the model checkpoint at deepset/minilm-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Question: How does Von Neuman's Computer Architecture use memory?
Answer:to store both the program and data.


In [11]:
idx = 30
question = mlqa['test']['question'][idx]
context = mlqa['test']['context'][idx]
answer = mlqa['test']['answers'][idx]

In [12]:
question, context, answer

('What must be ratified if a country wishes to join the WTO?',
 'After the Uruguay round, the GATT became the basis for the establishment of the World Trade Organization.  Because ratification of TRIPS is a compulsory requirement of World Trade Organization membership, any country seeking to obtain hard access to the numerous international markets opened by the World Trade Organization must enact the strict intellectual property laws mandated by TRIPS.  For this reason, TRIPS is the most important multilateral instrument for the globalization of intellectual property laws.  States like Russia and China, that were very unlikely to join the Berne Convention have found the prospect of WTO membership a powerful enticement.',
 {'answer_start': [131], 'text': ['TRIPS']})

In [13]:
qa_extractive(model_ckp, question, context)

Some weights of the model checkpoint at deepset/minilm-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Question: What must be ratified if a country wishes to join the WTO?
Answer:trips is a compulsory requirement of world trade organization membership, any country seeking to obtain hard access to the numerous international markets opened by the world trade organization must enact the strict intellectual property laws mandated


In [23]:
question = "In computer vision, should we rely only on CNNs models and why?"
context = """While the Transformer architecture has become
the de-facto standard for natural language processing tasks,
its applications to computer vision remain limited.
In vision, attention is either applied in conjunction with
convolutional networks, or used to replace certain components of
convolutional networks while keeping their overall structure in place.
We show that this reliance on CNNs is not necessary and a pure transformer
applied directly to sequences of image patches
can perform very well on image classification tasks.
When pre-trained on large amounts of data and transferred to
multiple mid-sized or small image recognition benchmarks
(ImageNet, CIFAR-100, VTAB, etc.), Vision Transformer (ViT) attains
excellent results compared to state-of-the-art convolutional
networks while requiring substantially fewer computational
resources to train.
"""

In [24]:
qa_extractive(model_ckp, question, context)

Some weights of the model checkpoint at deepset/minilm-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Question: In computer vision, should we rely only on CNNs models and why?
Answer:not necessary and


In [25]:
question = "In computer vision classification tasks how well can a pure transformer model perform?"
qa_extractive(model_ckp, question, context)

Some weights of the model checkpoint at deepset/minilm-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Question: In computer vision classification tasks how well can a pure transformer model perform?
Answer:very well on


In [26]:
question = "When do ViT perform well on image classification tasks"
qa_extractive(model_ckp, question, context)

Some weights of the model checkpoint at deepset/minilm-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Question: When do ViT perform well on image classification tasks
Answer:when pre - trained on large amounts of data and transferred to multiple mid - sized or small image recognition benchmarks (


In [19]:
question = "¿Qué es computación cuántica?"
context = """La computación cuántica o informática
cuántica​ es un paradigma de computación distinto al de la informática clásica.
Se basa en el uso de cúbits (qubits en inglés), una especial combinación de unos y ceros.
Los bits de la computación clásica pueden estar en 1 o en 0, pero solo un estado a la vez,
en tanto que el cúbit (quantum bit) puede tener los dos estados simultáneamente.
Esto da lugar a nuevas puertas lógicas que hacen posibles nuevos algoritmos."""

In [20]:
qa_extractive(model_ckp, question, context)

Some weights of the model checkpoint at deepset/minilm-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Question: ¿Qué es computación cuántica?
Answer:un paradigma de


In [21]:
question = "What is quantum computing?"
context = """A quantum computer is a computer that exploits quantum mechanical
 phenomena. On small scales, physical matter exhibits properties of both
 particles and waves, and quantum computing leverages this behavior using
 specialized hardware. Classical physics cannot explain the operation of
 these quantum devices, and a scalable quantum computer could perform
 some calculations exponentially faster[a] than any modern "classical" computer.
 Theoretically a large-scale quantum computer could break some widely used
 encryption schemes and aid physicists in performing physical simulations;
 however, the current state of the art is largely experimental and impractical,
 with several obstacles to useful applications.
"""

In [22]:
qa_extractive(model_ckp, question, context)

Some weights of the model checkpoint at deepset/minilm-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Question: What is quantum computing?
Answer:a computer that exploits quantum mechanical phenomena.


1. ¿Que limitaciones se encontraron?
- Se observa que las respuestas pueden ser incompletas o terminar con la conjunción and pero no decir nada más.
- Al usar pregunta y contexto tanto en español como en inglés se observa que hay un mejor rendimiento en inglés.
- El desempeño en el abstract del paper no fue el esperado ya que la respuesta fue muy corta si la pregunta no asemeja al contexto. Si la pregunta asemeja al contexto, el modelo funciona bastante bien.
- Parece que no se tiene control sobre la cantidad de palabras generadas en la respuesta
2. ¿Las respuestas fueron precisas?

- En la mayoría de casos sí sobre todo si la pregunta le permite a la IAG encontrar el contexto. Si la pregunta se vuelve más complicada (e.g. ¿por qué?) parece que el modelo no puede generar una respuesta completa.
- Es necesario que a pregunta contenga porciones del contexto para que la IA extractiva funcione adecuadamente

## Fine Tuning

Información adicional disponible en https://huggingface.co/docs/transformers/en/training

### Ejercicio Fine Tuning Distilbert en IMDB

Realizar fine-tuning con un modelo pre-entrenado DistilBERT usando el dataset de reseñas de películas IMDB para realizar clasificación de texto (positivo o negativo).

El modelo será ajustado con la biblioteca Hugging Face Transformers y los resultados serán evaluados.

Ajuste num_train_epochs y per_device_train_batch_size para observar cómo afecta al tiempo y a la precisión.

Introduzca tres reseñas propias para verificar si el modelo clasifica correctament las reseñas como positivas o negativas.

Evalúe el modelo en una muestra del dataset de prueba.

Muestre la predicción del modelo para sus propias reseñas. **texto en negrita**

In [2]:
!pip3 install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


#### Carga del Dataset

In [3]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

def tokenize(example):
  return tokenizer(example["text"], padding="max_length", truncation=True, max_length=512)

dataset = load_dataset("imdb")
tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 50000
    })
})

Reducimos el numero de ejemplos para economizar recursos y tiempo

In [16]:
small_train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_dataset["test"].shuffle(seed=42).select(range(1000))

In [17]:
small_train_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 1000
})

Controlando los parametros de entrenamiento

In [19]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="bert_finetuned",
                                  num_train_epochs=5,
                                  per_device_train_batch_size=32,
                                  per_device_eval_batch_size=32,
                                  warmup_steps=250,
                                  weight_decay=0.01,
                                  eval_strategy="epoch",
                                  eval_steps=100,
                                  save_steps=100,
                                  logging_dir="./logs")

Configurando la evaluación

In [10]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.9 MB/s eta 0:00:00


In [12]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

#### Entrenamiento

In [20]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [21]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.501921,0.902000
2,No log,0.499141,0.894000
3,No log,0.436315,0.909000
4,No log,0.456882,0.908000
5,No log,0.449916,0.905000


TrainOutput(global_step=160, training_loss=0.02724228799343109, metrics={'train_runtime': 342.9984, 'train_samples_per_second': 14.577, 'train_steps_per_second': 0.466, 'total_flos': 662336993280000.0, 'train_loss': 0.02724228799343109, 'epoch': 5.0})

In [25]:
example_input = tokenizer("Alien Covenant was a good movie but some things were repetitive", return_tensors="pt")
example_input = example_input.to(model.device)
output = model(**example_input)
predicted_label = torch.argmax(output.logits).item()
predicted_label

0

In [26]:
model.save_pretrained("./imdb_bert_finetuned")
tokenizer.save_pretrained("./imdb_bert_finetuned")

('./imdb_bert_finetuned/tokenizer_config.json',
 './imdb_bert_finetuned/special_tokens_map.json',
 './imdb_bert_finetuned/vocab.txt',
 './imdb_bert_finetuned/added_tokens.json',
 './imdb_bert_finetuned/tokenizer.json')

#### Evaluación de reseñas propias

In [27]:
reviews = [
    "This movie was fantastic! I loved the plot and the acting.",
    "I thought the movie was pretty average. It had its moments, but nothing special.",
    "Absolutely terrible. A complete waste of time and money."
]

In [28]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

imdb_model = AutoModelForSequenceClassification.from_pretrained("./imdb_bert_finetuned")
imdb_tokenizer = AutoTokenizer.from_pretrained("./imdb_bert_finetuned")

In [29]:
for review in reviews:
    review_input = imdb_tokenizer(review, return_tensors="pt")
    review_input = review_input.to(imdb_model.device)
    output = imdb_model(**review_input)
    predicted_label = torch.argmax(output.logits).item()
    print(f"Review: {review}\nPredicted Label: {'positive' if predicted_label == 1 else 'negative'}\n")

Review: This movie was fantastic! I loved the plot and the acting.
Predicted Label: positive

Review: I thought the movie was pretty average. It had its moments, but nothing special.
Predicted Label: negative

Review: Absolutely terrible. A complete waste of time and money.
Predicted Label: negative



#### ¿Qué clase de Fine Tuning hemos realizado?

Fine Tuning completo

## Zero Shot Clasification



In [31]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification")
model = "facebook/bart-large-mnli"
# texto de ejemplo
sequence_to_classify = "Las nuevas disposiciones de las autoridades son frustrantes"
candidate_labels = ['positivo', 'negativo', 'neutral']
results = classifier(sequence_to_classify, candidate_labels)
results

No model was supplied, defaulted to facebook/bart-large-mnli and revision d7645e1 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


{'sequence': 'Las nuevas disposiciones de las autoridades son frustrantes',
 'labels': ['negativo', 'positivo', 'neutral'],
 'scores': [0.9964271783828735, 0.002031923970207572, 0.001540849101729691]}

In [32]:
for label, score in zip(results['labels'], results['scores']):
  print(f"{label}: {score}")

negativo: 0.9964271783828735
positivo: 0.002031923970207572
neutral: 0.001540849101729691


Probando para varias reseñas

In [33]:
sequences  = ["La película de Alien Covenant era prometedora pero fue más de lo mismo",
              "Scary Movie es sólo sangre y nada de historia",
              "La película Goten Tag Ramon fue excelente"]

for sequence in sequences:
  results = classifier(sequence, candidate_labels)
  print(f"Sequence: {sequence}")
  for label, score in zip(results['labels'], results['scores']):
    print(f"{label}: {score}")


Sequence: La película de Alien Covenant era prometedora pero fue más de lo mismo
negativo: 0.8942618370056152
positivo: 0.06849803030490875
neutral: 0.037240151315927505
Sequence: Scary Movie es sólo sangre y nada de historia
negativo: 0.7426891326904297
neutral: 0.25257593393325806
positivo: 0.004734946880489588
Sequence: La película Goten Tag Ramon fue excelente
positivo: 0.9970551133155823
negativo: 0.0016250016633421183
neutral: 0.0013198803644627333


## QA Answering Zero Shot

In [35]:
from transformers import pipeline
# Cargar el pipeline de Question Answering
qa_pipeline = pipeline("question-answering",
model="deepset/roberta-base-squad2")
# Contexto y pregunta
context = "Kimchi is a traditional Korean dish made of fermented vegetables, typically napa cabbage and radishes."
question = "What is kimchi made of?"
# Realizar inferencia
result = qa_pipeline(question=question, context=context)
# Mostrar respuesta
print("Pregunta:", question)
print("Respuesta:", result['answer'])


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cuda:0


Pregunta: What is kimchi made of?
Respuesta: fermented vegetables


In [36]:
question = "What is quantum computing?"
context = """A quantum computer is a computer that exploits quantum mechanical
 phenomena. On small scales, physical matter exhibits properties of both
 particles and waves, and quantum computing leverages this behavior using
 specialized hardware. Classical physics cannot explain the operation of
 these quantum devices, and a scalable quantum computer could perform
 some calculations exponentially faster[a] than any modern "classical" computer.
 Theoretically a large-scale quantum computer could break some widely used
 encryption schemes and aid physicists in performing physical simulations;
 however, the current state of the art is largely experimental and impractical,
 with several obstacles to useful applications.
"""

result = qa_pipeline(question=question, context=context)
# Mostrar respuesta
print("Pregunta:", question)
print("Respuesta:", result['answer'])

Pregunta: What is quantum computing?
Respuesta: a computer that exploits quantum mechanical
 phenomena


In [38]:
question = "In computer vision, should we rely only on CNNs models?"
context = """While the Transformer architecture has become
the de-facto standard for natural language processing tasks,
its applications to computer vision remain limited.
In vision, attention is either applied in conjunction with
convolutional networks, or used to replace certain components of
convolutional networks while keeping their overall structure in place.
We show that this reliance on CNNs is not necessary and a pure transformer
applied directly to sequences of image patches
can perform very well on image classification tasks.
When pre-trained on large amounts of data and transferred to
multiple mid-sized or small image recognition benchmarks
(ImageNet, CIFAR-100, VTAB, etc.), Vision Transformer (ViT) attains
excellent results compared to state-of-the-art convolutional
networks while requiring substantially fewer computational
resources to train.
"""

result = qa_pipeline(question=question, context=context)
# Mostrar respuesta
print("Pregunta:", question)
print("Respuesta:", result['answer'])

Pregunta: In computer vision, should we rely only on CNNs models?
Respuesta: not necessary


In [39]:
question = "What are the benefits of using vision transformers instead of CNN models in computer vision?"
context = """While the Transformer architecture has become
the de-facto standard for natural language processing tasks,
its applications to computer vision remain limited.
In vision, attention is either applied in conjunction with
convolutional networks, or used to replace certain components of
convolutional networks while keeping their overall structure in place.
We show that this reliance on CNNs is not necessary and a pure transformer
applied directly to sequences of image patches
can perform very well on image classification tasks.
When pre-trained on large amounts of data and transferred to
multiple mid-sized or small image recognition benchmarks
(ImageNet, CIFAR-100, VTAB, etc.), Vision Transformer (ViT) attains
excellent results compared to state-of-the-art convolutional
networks while requiring substantially fewer computational
resources to train.
"""

result = qa_pipeline(question=question, context=context)
# Mostrar respuesta
print("Pregunta:", question)
print("Respuesta:", result['answer'])

Pregunta: What are the benefits of using vision transformers instead of CNN models in computer vision?
Respuesta: can perform very well on image classification tasks


### ¿Cuál es la diferencia con fine Tuneing?
En fine tuning se realiza el ajuste de pesos lo que requiere usar recursos computacionales altos. Aquí se obtiene un resultado muy adecuado sin incurrir en costos computacionales de entrenamiento de modelos. Pero podría compremeter a la precisión.

## Few Shot Learning

In [46]:
from transformers import pipeline
# Cargar pipeline de generación de texto con un modelo GPT
generator = pipeline("text-generation",
model="EleutherAI/gpt-neo-1.3B")
# Prompt con ejemplos para few-shot
prompt = """
 Classify the following text into Positive, Negative, or Neutral
sentiment.
 Example 1:
 Text: "I love this product! It's amazing."
 Sentiment: Positive
 Example 2:
 Text: "This is the worst experience I've ever had."
 Sentiment: Negative
 Example 3:
 Text: "The product is okay. Not great, not bad."
 Sentiment: Neutral
 Now classify this:
 Text: "The service was excellent, and I will definitely return."
 Sentiment:
 """


Device set to use cuda:0


In [49]:
# Generar texto
result = generator(prompt, max_length=150,
                   num_return_sequences=1)

result[0]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{'generated_text': '\n Classify the following text into Positive, Negative, or Neutral \nsentiment.\n Example 1:\n Text: "I love this product! It\'s amazing."\n Sentiment: Positive\n Example 2:\n Text: "This is the worst experience I\'ve ever had."\n Sentiment: Negative\n Example 3:\n Text: "The product is okay. Not great, not bad."\n Sentiment: Neutral\n Now classify this:\n Text: "The service was excellent, and I will definitely return."\n Sentiment:\n  Negative\n\n'}

In [50]:
# Mostrar resultado
print("Resultado Few-Shot:")
print(result[0]["generated_text"])

Resultado Few-Shot:

 Classify the following text into Positive, Negative, or Neutral 
sentiment.
 Example 1:
 Text: "I love this product! It's amazing."
 Sentiment: Positive
 Example 2:
 Text: "This is the worst experience I've ever had."
 Sentiment: Negative
 Example 3:
 Text: "The product is okay. Not great, not bad."
 Sentiment: Neutral
 Now classify this:
 Text: "The service was excellent, and I will definitely return."
 Sentiment:
  Negative




Probando unos ejemplos del caso de Fine Tuning

In [51]:
for review in reviews:
  prompt = "Classify the following text into Positive, Negative, or Neutral sentiment.\n"
  prompt += f"Text: {review}\n"
  prompt += "Sentiment:"
  print(prompt)
  result = generator(prompt, max_length=150, )
  print("Resultado Few-Shot:")
  print(result[0]["generated_text"])
  print(f"----------------------------------------")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Classify the following text into Positive, Negative, or Neutral sentiment.
Text: This movie was fantastic! I loved the plot and the acting.
Sentiment:


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Resultado Few-Shot:
Classify the following text into Positive, Negative, or Neutral sentiment.
Text: This movie was fantastic! I loved the plot and the acting.
Sentiment: Neutral

Question 4 of 11

How do we tell which words are more meaningful in a sentence?

Use this analysis to find out which words are more meaningful in a text. In a sentence,
what words carry most information? Find similar words, such as synonyms and antonyms. Then
compare where the most words are used. Take a step back and think about how the words
are grouped into the sentences. Find out which group, or sets of words, are most relevant to the idea
or subject you are trying to describe
----------------------------------------
Classify the following text into Positive, Negative, or Neutral sentiment.
Text: I thought the movie was pretty average. It had its moments, but nothing special.
Sentiment:


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Resultado Few-Shot:
Classify the following text into Positive, Negative, or Neutral sentiment.
Text: I thought the movie was pretty average. It had its moments, but nothing special.
Sentiment: Negative
Rating: 2.5/5

There are things in life that shouldn’t be allowed to go by without being noticed. That’s why we at Positively Positive are dedicated to the fight against any and all prejudice in our community. Our goal isn’t to let it go by unnoticed. We want to bring about a change in this world that is positive for everyone.

Our vision is to bring about positive change with respect to the way people are treated in our communities and to help others be positively positively positive
----------------------------------------
Classify the following text into Positive, Negative, or Neutral sentiment.
Text: Absolutely terrible. A complete waste of time and money.
Sentiment:
Resultado Few-Shot:
Classify the following text into Positive, Negative, or Neutral sentiment.
Text: Absolutely terrib

Se observa que existe una gran precisión al usar Fine Tuning. Zero Shot Classification funciona bastante bien pero few shot learning depende del prompt para realizar un trabajo adecuado.